In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hindienglish-corpora/Hindi_English_Truncated_Corpus.csv


In [2]:
from numpy import array, argmax, random, take
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Bidirectional, RepeatVector, TimeDistributed,Masking
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras import optimizers
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
pd.set_option('display.max_colwidth', 200)

Using TensorFlow backend.


In [3]:
df=pd.read_csv("/kaggle/input/hindienglish-corpora/Hindi_English_Truncated_Corpus.csv")
df

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what needs to be done.,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह करने कि अनुमति नहीं है ."
1,ted,"I'd like to tell you about one such child,","मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी,"
2,indic2012,This percentage is even greater than the percentage in India.,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not paying attention.,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called Upanishad.,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।
...,...,...,...
127602,indic2012,Examples of art deco construction can be found around Marine Drive and Oval Ground areas.,आर्ट डेको शैली के निर्माण मैरीन ड्राइव और ओवल मैदान के किनारे दिखाई देते हैं।
127603,ted,and put it in our cheeks.,और अपने गालों में डाल लेते हैं।
127604,tides,"As for the other derivatives of sulphur , the country 's needs of iron , copper , sodium , etc . sulphates were limited , and the production achieved after the war was generally adequate .","जहां तक गंधक के अन्य उत्पादों का प्रश्न है , देश में लोहे , तांबे , सोडियम , सल्फेट आदि की आवश्यकता सीमित थी और युद्धोपरांत हुआ उत्पादन सामान्य रूप से पर्याप्त था ."
127605,tides,its complicated functioning is defined thus in a popular riddle :,Zरचना-प्रकिया को उसने एक पहेली में यों बांधा है .


In [4]:
df.isnull().sum()

source              0
english_sentence    2
hindi_sentence      0
dtype: int64

In [5]:
df.shape

(127607, 3)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127607 entries, 0 to 127606
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   source            127607 non-null  object
 1   english_sentence  127605 non-null  object
 2   hindi_sentence    127607 non-null  object
dtypes: object(3)
memory usage: 2.9+ MB


In [7]:
df.describe()

,source,english_sentence,hindi_sentence
count,127607,127605,127607
unique,3,124317,97662
top,tides,(Laughter),(हँसी)
freq,50000,555,212


# As we see that this dataset is very big so let us take 30000 rows from this dataset..

In [8]:
df.head(10)

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what needs to be done.,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह करने कि अनुमति नहीं है ."
1,ted,"I'd like to tell you about one such child,","मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी,"
2,indic2012,This percentage is even greater than the percentage in India.,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not paying attention.,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called Upanishad.,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।
5,tides,"The then Governor of Kashmir resisted transfer , but was finally reduced to subjection with the aid of British .","कश्मीर के तत्कालीन गवर्नर ने इस हस्तांतरण का विरोध किया था , लेकिन अंग्रेजों की सहायता से उनकी आवाज दबा दी गयी ."
6,indic2012,In this lies the circumstances of people before you.,इसमें तुमसे पूर्व गुज़रे हुए लोगों के हालात हैं।
7,ted,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
8,indic2012,“”Global Warming“” refer to warming caused in recent decades and probability of its continual presence and its indirect effect on human being.,ग्लोबल वॉर्मिंग से आशय हाल ही के दशकों में हुई वार्मिंग और इसके निरंतर बने रहने के अनुमान और इसके अप्रत्यक्ष रूप से मानव पर पड़ने वाले प्रभाव से है।
9,tides,You may want your child to go to a school that is not run by the LEA - a non-maintained special school or an independent school that can meet your child 's needs .,"हो सकता है कि आप चाहते हों कि आप का नऋर्नमेनटेन्ड ह्यबिना किसी समर्थन के हृ विशेष स्कूल , या किसी स्वतंत्र स्कूल में जाए , इजसके पास विशेष शैक्षणिक जऋऋरतों वाले बच्चों के प्रति सहूलियत हों . ."


In [9]:
new_df=df.sample(n = 30000,random_state=42) 
new_df

,source,english_sentence,hindi_sentence
3556,tides,He declares the result and reports it to the Election Commission and to the Secretary-General of the concerned House .,वही परिणाम की घोषणा करता है और निर्वाचन आयोग को और संबद्ध सदन के महासचिव को उसकी सूचना देता है .
25899,ted,was a little uncomfortable for them.,थोडा कठिन था।
89038,indic2012,But Mulla Assamudin was proved to be not eligible.,मगर मुल्ला असमुद्दीन अक्षम सिद्ध हुए।
78212,ted,I would never have to make a book and then present it to a gallery,मुझे कभी भी किताब बना कर किसी प्रदर्शन-स्थल को देने की ज़रुरत नहीं पड़ी
96955,indic2012,headind kaun banega crorepati,शीर्षक कौन बनेगा करोड़पति (Kaun Banega Crorepati).
...,...,...,...
15190,ted,"and the story doesn't work if I tell you about her before,",और कहानी का मज़ा तभी है जब मैं उसके बारे में यहाँ तक आपको कुछ न बताऊँ.
37686,ted,"Then there's another group, bigger group.","फिर एक और समूह है, बड़ा समूह."
77404,tides,Today 's teenager does not mind wearing it with her denim jeans . ”,आज की किशोरियां तो जीन्स के साथ भी इन्हें पहनने लगी हैं . ' '
88546,tides,Q . Do you think Joshi should bring about further corrections in the field of education ?,> क्या आप समज्ह्ते हैं कि जोशी को शिक्षा के क्षेत्र में और सुधार लने चाहिए ?


In [10]:
new_df=new_df.reset_index()

In [11]:
new_df.head()

,index,source,english_sentence,hindi_sentence
0,3556,tides,He declares the result and reports it to the Election Commission and to the Secretary-General of the concerned House .,वही परिणाम की घोषणा करता है और निर्वाचन आयोग को और संबद्ध सदन के महासचिव को उसकी सूचना देता है .
1,25899,ted,was a little uncomfortable for them.,थोडा कठिन था।
2,89038,indic2012,But Mulla Assamudin was proved to be not eligible.,मगर मुल्ला असमुद्दीन अक्षम सिद्ध हुए।
3,78212,ted,I would never have to make a book and then present it to a gallery,मुझे कभी भी किताब बना कर किसी प्रदर्शन-स्थल को देने की ज़रुरत नहीं पड़ी
4,96955,indic2012,headind kaun banega crorepati,शीर्षक कौन बनेगा करोड़पति (Kaun Banega Crorepati).


In [12]:
new_df['english_sentence'][11]

'What if I need general information about applying for a work permit ?'

In [13]:
new_df.shape

(30000, 4)

In [14]:
new_df.dtypes

index                int64
source              object
english_sentence    object
hindi_sentence      object
dtype: object

# Now cleaning the sentences..

In [15]:
new_df['english_sentence'] = new_df['english_sentence'].astype(str)

In [16]:
# Lowercase all characters
new_df['english_sentence']=new_df['english_sentence'].apply(lambda x: x.lower())
new_df['hindi_sentence']=new_df['hindi_sentence'].apply(lambda x: x.lower())

In [17]:
import  re
import string
from string import digits

In [18]:
new_df['hindi_sentence']=new_df['hindi_sentence'].apply(lambda x: re.sub("'", '', x))
new_df['english_sentence']=new_df['english_sentence'].apply(lambda x: re.sub("'", '', x))
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
new_df['hindi_sentence']=new_df['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
new_df['english_sentence']=new_df['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [19]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
new_df['english_sentence']=new_df['english_sentence'].apply(lambda x: x.translate(remove_digits))
new_df['hindi_sentence']=new_df['hindi_sentence'].apply(lambda x: x.translate(remove_digits))

new_df['hindi_sentence'] = new_df['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
new_df['english_sentence']=new_df['english_sentence'].apply(lambda x: x.strip())
new_df['hindi_sentence']=new_df['hindi_sentence'].apply(lambda x: x.strip())
new_df['english_sentence']=new_df['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
new_df['hindi_sentence']=new_df['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))

# Adding START and END tokens to the target sequences(i.e Hindi Sentences):- 

In [20]:
new_df['hindi_sentence']=new_df['hindi_sentence'].apply(lambda x:'START_ ' + x + ' _END') 

In [21]:
new_df.head()

,index,source,english_sentence,hindi_sentence
0,3556,tides,he declares the result and reports it to the election commission and to the secretarygeneral of the concerned house,START_ वही परिणाम की घोषणा करता है और निर्वाचन आयोग को और संबद्ध सदन के महासचिव को उसकी सूचना देता है _END
1,25899,ted,was a little uncomfortable for them,START_ थोडा कठिन था। _END
2,89038,indic2012,but mulla assamudin was proved to be not eligible,START_ मगर मुल्ला असमुद्दीन अक्षम सिद्ध हुए। _END
3,78212,ted,i would never have to make a book and then present it to a gallery,START_ मुझे कभी भी किताब बना कर किसी प्रदर्शनस्थल को देने की ज़रुरत नहीं पड़ी _END
4,96955,indic2012,headind kaun banega crorepati,START_ शीर्षक कौन बनेगा करोड़पति kaun banega crorepati _END


# Now calculating the english and hindi sentence lengths..

In [22]:
senlen1=[]

In [23]:
for i in new_df['english_sentence']:
    res = len(i.split()) 
    senlen1.append(res)

In [24]:
# senlen1

In [25]:
new_df['engsentlen']=senlen1

In [26]:
new_df.head()

,index,source,english_sentence,hindi_sentence,engsentlen
0,3556,tides,he declares the result and reports it to the election commission and to the secretarygeneral of the concerned house,START_ वही परिणाम की घोषणा करता है और निर्वाचन आयोग को और संबद्ध सदन के महासचिव को उसकी सूचना देता है _END,19
1,25899,ted,was a little uncomfortable for them,START_ थोडा कठिन था। _END,6
2,89038,indic2012,but mulla assamudin was proved to be not eligible,START_ मगर मुल्ला असमुद्दीन अक्षम सिद्ध हुए। _END,9
3,78212,ted,i would never have to make a book and then present it to a gallery,START_ मुझे कभी भी किताब बना कर किसी प्रदर्शनस्थल को देने की ज़रुरत नहीं पड़ी _END,15
4,96955,indic2012,headind kaun banega crorepati,START_ शीर्षक कौन बनेगा करोड़पति kaun banega crorepati _END,4


In [27]:
len(new_df['english_sentence'])

30000

In [28]:
senlen2=[]

In [29]:
for j in new_df['hindi_sentence']:
     res = len(j.split())
     senlen2.append(res)

In [30]:
# senlen2

In [31]:
new_df['hinsenlen']=senlen2

In [32]:
new_df.head(10)

,index,source,english_sentence,hindi_sentence,engsentlen,hinsenlen
0,3556,tides,he declares the result and reports it to the election commission and to the secretarygeneral of the concerned house,START_ वही परिणाम की घोषणा करता है और निर्वाचन आयोग को और संबद्ध सदन के महासचिव को उसकी सूचना देता है _END,19,22
1,25899,ted,was a little uncomfortable for them,START_ थोडा कठिन था। _END,6,5
2,89038,indic2012,but mulla assamudin was proved to be not eligible,START_ मगर मुल्ला असमुद्दीन अक्षम सिद्ध हुए। _END,9,8
3,78212,ted,i would never have to make a book and then present it to a gallery,START_ मुझे कभी भी किताब बना कर किसी प्रदर्शनस्थल को देने की ज़रुरत नहीं पड़ी _END,15,16
4,96955,indic2012,headind kaun banega crorepati,START_ शीर्षक कौन बनेगा करोड़पति kaun banega crorepati _END,4,9
5,317,tides,no other national leader except nehru shared that concern,START_ नेहरू को छोड़कर और किसी भी राष्ट्र नेता के मन में यह चिंता नहीं थे _END,9,17
6,109185,indic2012,bachchan continued togive their support to the samajvadi party which included political propaganda and promotions,START_ बच्चन ने समाजवादी पार्टी के लिए अपना समर्थन देना जारी रखा जिसमें राजनैतिक अभियान अर्थात प्रचार प्रसार करना शामिल था। _END,15,22
7,117804,tides,innocent people were shot arrested jailed and tortured by the british rulers,START_ अंग्रेज शासकों ने निर्दोष लोगों की जान ली उन्हें पकड़कर जेलों में बंद किया और यातनाएं दीं _END,12,19
8,126165,tides,it means that fascism will make a triumphant attempt at world dominion in cooperation with the fascism of japan,START_ इसका मतलब है कि फासिस्टवाद जापान के फासिस्टवाद के साथ मिलकर सारी दुनिया पर कामयाबी के साथ कब्जा करने की कोशिश करेगा _END,19,24
9,37263,ted,just as the poets and writers described,START_ जैसे कवियों और लेखकों ने वर्णन किया है _END,7,10


Our dataset is now cleaned..

In [33]:
new_df=new_df[new_df['engsentlen']<=10]
new_df=new_df[new_df['hinsenlen']<=10]

In [34]:
new_df.head()

,index,source,english_sentence,hindi_sentence,engsentlen,hinsenlen
1,25899,ted,was a little uncomfortable for them,START_ थोडा कठिन था। _END,6,5
2,89038,indic2012,but mulla assamudin was proved to be not eligible,START_ मगर मुल्ला असमुद्दीन अक्षम सिद्ध हुए। _END,9,8
4,96955,indic2012,headind kaun banega crorepati,START_ शीर्षक कौन बनेगा करोड़पति kaun banega crorepati _END,4,9
9,37263,ted,just as the poets and writers described,START_ जैसे कवियों और लेखकों ने वर्णन किया है _END,7,10
10,8579,tides,the rich flora of central america is under threat,START_ मध्य अमेरिकी बहुमूल्य वनस्पति खतरे में है _END,9,9


In [35]:
new_df.shape

(8998, 6)

In [36]:
print("Maximum length of English Sentence ",max(new_df['engsentlen']))
print("Maximum length of Hindi Sentence ",max(new_df['hinsenlen']))

Maximum length of English Sentence  10
Maximum length of Hindi Sentence  10


In [37]:
max_length_eng=max(new_df['engsentlen'])
max_length_hin=max(new_df['hinsenlen'])

In [38]:
def tokenization(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [39]:
english_tokenizer = tokenization(new_df['english_sentence'].values)

# Creating english and hindi vocabulary:-

In [40]:
english_words=set()
for eng in new_df['english_sentence']:
    for word in eng.split():
        if word not in english_words:
            english_words.add(word)

hindi_words=set()
for hin in new_df['hindi_sentence']:
    for word in hin.split():
        if word not in hindi_words:
            hindi_words.add(word)

In [41]:
english_words.add(' ')

In [42]:
hindi_words.add(' ')

In [43]:
len(english_words)

8358

In [44]:
len(hindi_words)

9129

In [45]:
input_texts = []
target_texts = []

# Making the seperate lists of English and Hindi Sentences:-

In [46]:
for i in new_df['english_sentence']:
    input_texts.append(i)

In [47]:
# input_texts

In [48]:
len(input_texts)

8998

In [49]:
for i in new_df['hindi_sentence']:
    target_texts.append(i)

In [50]:
# target_texts

In [51]:
num_encoder_tokens = len(english_words)
num_decoder_tokens = len(hindi_words)

# Created a Dictionary to convert a all the words to integer:-

In [52]:
input_words = sorted(list(english_words))
target_words = sorted(list(hindi_words))


input_token_index = dict(
    [(char, i) for i, char in enumerate(input_words)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_words)])

In [53]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [54]:
data=np.zeros((2,3),dtype='float32')
data

array([[0., 0., 0.],
       [0., 0., 0.]], dtype=float32)

In [55]:
data=np.zeros((2,2,4),dtype='float32')
data

array([[[0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.]]], dtype=float32)

In [56]:
data=np.zeros((2,10,6),dtype='float32')
data

array([[[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]]], dtype=float32)

# So According to this we will encode the data as input by converting it into 3D Numpy Array. So according to our observations we will set 1st parameter as the total input data, 2nd as the max length among all the sentences and the 3rd as the dimension as each word will be represented by a number between 0 to the last index of word present..

In [57]:
encoder_input_data = np.zeros(
    (len(input_texts), max_length_eng, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_length_hin, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts),  max_length_hin, num_decoder_tokens),
    dtype='float32')

In [58]:
encoder_input_data

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

# encoder_input_data is a 3D array of shape (num_pairs, max_english_sentence_length, num_english_characters) containing a one-hot vectorization of the English sentences.

In [59]:
encoder_input_data.shape

(8998, 10, 8358)

# decoder_input_data is a 3D array of shape (num_pairs, max_hindi_sentence_length, num_hindi_characters) containg a one-hot vectorization of the Hindi sentences.

In [60]:
decoder_input_data.shape

(8998, 10, 9129)

# decoder_target_data is the same as decoder_input_data but offset by one timestep. decoder_target_data[:, t, :] will be the same as decoder_input_data[:, t + 1, :].

In [61]:
decoder_target_data.shape

(8998, 10, 9129)

In [62]:
text='This is Rahul'

In [63]:
for word in text.split():
    print(word)

This
is
Rahul


In [64]:
input_token_index


{' ': 0,
 'a': 1,
 'aadam': 2,
 'aaen': 3,
 'aagra': 4,
 'aainaeakbary': 5,
 'aaj': 6,
 'aalekh': 7,
 'aambat': 8,
 'aaranya': 9,
 'aarati': 10,
 'aarun': 11,
 'aayatstext': 12,
 'ab': 13,
 'abbas': 14,
 'abdul': 15,
 'abhinav': 16,
 'abidjan': 17,
 'ability': 18,
 'able': 19,
 'ablest': 20,
 'abnormal': 21,
 'abnormalities': 22,
 'abode': 23,
 'about': 24,
 'above': 25,
 'abraham': 26,
 'abrahamic': 27,
 'abridged': 28,
 'absconded': 29,
 'absolute': 30,
 'absolutely': 31,
 'absorb': 32,
 'abstract': 33,
 'abu': 34,
 'abual': 35,
 'abuse': 36,
 'academic': 37,
 'academy': 38,
 'acceleration': 39,
 'accept': 40,
 'acceptance': 41,
 'accepted': 42,
 'access': 43,
 'accident': 44,
 'accompanied': 45,
 'accompanying': 46,
 'according': 47,
 'account': 48,
 'accountability': 49,
 'accountant': 50,
 'accounted': 51,
 'accumulated': 52,
 'accumulating': 53,
 'accused': 54,
 'aceitha': 55,
 'achaantar': 56,
 'achieve': 57,
 'achieved': 58,
 'achieves': 59,
 'acid': 60,
 'acolytes': 61,
 'acor

In [65]:
input_token_index['poets']

5513

In [66]:
len(input_token_index)

8358

# Doing One Hot Encoding of All the Three 3D Arrays:-

In [67]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    # Loop over each char in an input text
        for t, word in enumerate(input_text.split()):
                    #print(word,t)
                    encoder_input_data[i, t, input_token_index[word]]=1. #input token index gives index from the dictionary created
                    
        encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.            
        for t, word in enumerate(target_text.split()):
#                     print(t,word)
                    if t<len(target_text.split())-1: # We are subtracting 1 as we are not taking the START Keyword at the starting.
                        decoder_input_data[i, t,target_token_index[word]]=1. #input token index gives index from the dictionary created of target sequence
                   
                    if t>0: 
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep So we will not include t=0th word. We will take words after that.
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
                        
        decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
        decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [68]:
encoder_input_data

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0.

In [69]:
decoder_input_data

array([[[0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0.

In [70]:
decoder_target_data

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0.

# Encoder Decoder Architecture:-

In [71]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

In [72]:
encoder_inputs=Input(shape=(None, num_encoder_tokens))
encoder=LSTM(256,return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
#here we will ignore the encoder outputs and take only the two states.
encoder_states=[state_h,state_c]

In [73]:
encoder_states

[<tf.Tensor 'lstm_1/while:4' shape=(None, 256) dtype=float32>,
 <tf.Tensor 'lstm_1/while:5' shape=(None, 256) dtype=float32>]

In [74]:
# Setting up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We are setting up our decoder to return full output sequences,
# and to return internal states as well. We are not using the 
# return states in the training model, but we will use them in inference.

decoder = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Defining the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Model Compilation and Training:-

In [75]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy')

In [76]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 8358)   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 9129)   0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 8821760     input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  9611264     input_2[0][0]                    
                                                                 lstm_1[0][1]               

In [77]:
model.fit([encoder_input_data,decoder_input_data],decoder_target_data,batch_size=128,epochs=2,validation_split=0.2)

Train on 7198 samples, validate on 1800 samples
Epoch 1/2
7198/7198 [==============================] - 21s 3ms/step - loss: 4.7716 - val_loss: 4.2437
Epoch 2/2
7198/7198 [==============================] - 16s 2ms/step - loss: 4.0807 - val_loss: 4.2089


In [78]:
model.save('final_model')

In [79]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [91]:
target_token_index

{' ': 0,
 'START_': 1,
 '_END': 2,
 'a': 3,
 'actuators': 4,
 'afternoon': 5,
 'aids': 6,
 'air': 7,
 'aks': 8,
 'ammonia': 9,
 'anand': 10,
 'ap': 11,
 'argentina': 12,
 'articulated': 13,
 'available': 14,
 'banega': 15,
 'beehive': 16,
 'behavior': 17,
 'best': 18,
 'bg': 19,
 'biotechnology': 20,
 'births': 21,
 'box': 22,
 'bridge': 23,
 'browse': 24,
 'burner': 25,
 'ca': 26,
 'cablesuspended': 27,
 'clean': 28,
 'climatecrisisnet': 29,
 'co': 30,
 'control': 31,
 'copd': 32,
 'crorepati': 33,
 'degreesoffreedom': 34,
 'dictionary': 35,
 'dimming': 36,
 'drivegrip': 37,
 'dynamic': 38,
 'english': 39,
 'experimental': 40,
 'facebook': 41,
 'false': 42,
 'faq': 43,
 'feed': 44,
 'flag': 45,
 'foundation': 46,
 'foursquare': 47,
 'function': 48,
 'g': 49,
 'gartner': 50,
 'global': 51,
 'globaloney': 52,
 'gogh': 53,
 'gps': 54,
 'hindi': 55,
 'hiv': 56,
 'html': 57,
 'hyper': 58,
 'i': 59,
 'icyball': 60,
 'impass': 61,
 'index': 62,
 'insula': 63,
 'intelligent': 64,
 'internatio

In [98]:
reverse_target_char_index

{0: ' ',
 1: 'START_',
 2: '_END',
 3: 'a',
 4: 'actuators',
 5: 'afternoon',
 6: 'aids',
 7: 'air',
 8: 'aks',
 9: 'ammonia',
 10: 'anand',
 11: 'ap',
 12: 'argentina',
 13: 'articulated',
 14: 'available',
 15: 'banega',
 16: 'beehive',
 17: 'behavior',
 18: 'best',
 19: 'bg',
 20: 'biotechnology',
 21: 'births',
 22: 'box',
 23: 'bridge',
 24: 'browse',
 25: 'burner',
 26: 'ca',
 27: 'cablesuspended',
 28: 'clean',
 29: 'climatecrisisnet',
 30: 'co',
 31: 'control',
 32: 'copd',
 33: 'crorepati',
 34: 'degreesoffreedom',
 35: 'dictionary',
 36: 'dimming',
 37: 'drivegrip',
 38: 'dynamic',
 39: 'english',
 40: 'experimental',
 41: 'facebook',
 42: 'false',
 43: 'faq',
 44: 'feed',
 45: 'flag',
 46: 'foundation',
 47: 'foursquare',
 48: 'function',
 49: 'g',
 50: 'gartner',
 51: 'global',
 52: 'globaloney',
 53: 'gogh',
 54: 'gps',
 55: 'hindi',
 56: 'hiv',
 57: 'html',
 58: 'hyper',
 59: 'i',
 60: 'icyball',
 61: 'impass',
 62: 'index',
 63: 'insula',
 64: 'intelligent',
 65: 'intern

In [103]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['START_']] = 1.
    

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        print(output_tokens)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        print(sampled_token_index)
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > max_length_hin):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [105]:
for seq_index in range(400):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

[[[6.3410832e-04 1.4585761e-06 2.4121257e-03 ... 2.3722503e-05
   3.8347640e-05 3.8523714e-05]]]
1378
[[[6.7920831e-04 2.2028401e-06 4.0888498e-03 ... 3.1667234e-05
   4.8824095e-05 4.7148034e-05]]]
1378
[[[1.5681079e-03 1.7502859e-06 1.1298175e-02 ... 2.8613837e-05
   4.5602585e-05 3.9252209e-05]]]
1879
[[[4.4865441e-03 1.5877856e-06 2.7774405e-02 ... 2.5483736e-05
   4.0365831e-05 3.3070748e-05]]]
1879
[[[1.4060232e-02 1.6394400e-06 6.0536746e-02 ... 2.2875149e-05
   3.4915302e-05 2.8505801e-05]]]
2
-
Input sentence: was a little uncomfortable for them
Decoded sentence: औरऔरकेके_END
[[[6.5568340e-04 1.5076747e-06 2.5512232e-03 ... 2.4342478e-05
   3.9382423e-05 3.9328286e-05]]]
1378
[[[7.9953752e-04 2.1638718e-06 4.9793753e-03 ... 3.1587682e-05
   4.8910337e-05 4.6517420e-05]]]
1378
[[[2.1228318e-03 1.7150455e-06 1.4804194e-02 ... 2.7932505e-05
   4.4366036e-05 3.8044669e-05]]]
1879
[[[6.6292295e-03 1.6485095e-06 3.6794223e-02 ... 2.4899595e-05
   3.8967599e-05 3.2171884e-05]]]
2
-
I

[[[2.4596031e-03 1.7006112e-06 1.7035691e-02 ... 2.7493421e-05
   4.3700329e-05 3.7420694e-05]]]
1879
[[[7.9375375e-03 1.6642227e-06 4.2053100e-02 ... 2.4424693e-05
   3.8120044e-05 3.1600805e-05]]]
2
-
Input sentence: he campaigned against untouchability
Decoded sentence: औरऔरके_END
[[[6.3836091e-04 1.4282272e-06 2.3682339e-03 ... 2.3371982e-05
   3.7743059e-05 3.8067192e-05]]]
1378
[[[6.3983083e-04 2.2170886e-06 3.6930868e-03 ... 3.1629355e-05
   4.8558042e-05 4.7428897e-05]]]
1378
[[[1.3417822e-03 1.7847487e-06 9.6672159e-03 ... 2.9016088e-05
   4.6219833e-05 4.0071562e-05]]]
1879
[[[3.6329667e-03 1.5630775e-06 2.3611197e-02 ... 2.5849915e-05
   4.1167164e-05 3.3594479e-05]]]
1879
[[[1.1048288e-02 1.6141058e-06 5.1680904e-02 ... 2.3446832e-05
   3.6092384e-05 2.9271407e-05]]]
2
-
Input sentence: and then i started progressing to other things
Decoded sentence: औरऔरकेके_END
[[[2.2611402e-03 2.1278092e-06 9.7209532e-03 ... 3.0415042e-05
   4.7074034e-05 4.5244524e-05]]]
1378
[[[7.49793

2
-
Input sentence: you could abuse it
Decoded sentence: औरऔरकेके_END
[[[6.4176763e-04 1.4783224e-06 2.4654004e-03 ... 2.3971375e-05
   3.8748432e-05 3.8843147e-05]]]
1378
[[[7.2898204e-04 2.1838218e-06 4.4591697e-03 ... 3.1619136e-05
   4.8878312e-05 4.6849804e-05]]]
1378
[[[1.8006630e-03 1.7264830e-06 1.2802962e-02 ... 2.8242810e-05
   4.5014171e-05 3.8622849e-05]]]
1879
[[[5.3664893e-03 1.6134112e-06 3.1684022e-02 ... 2.5185700e-05
   3.9731516e-05 3.2650991e-05]]]
1879
[[[1.7107222e-02 1.6499718e-06 6.8518698e-02 ... 2.2359709e-05
   3.3866669e-05 2.7774906e-05]]]
2
-
Input sentence: like saraswathi bharav shaurya and ganesh kunda etc
Decoded sentence: औरऔरकेके_END
[[[6.4793677e-04 1.4765625e-06 2.4710053e-03 ... 2.3971445e-05
   3.8774622e-05 3.8833463e-05]]]
1378
[[[7.2906556e-04 2.1842079e-06 4.4353958e-03 ... 3.1624473e-05
   4.8861155e-05 4.6867513e-05]]]
1378
[[[1.7835350e-03 1.7318596e-06 1.2662937e-02 ... 2.8316830e-05
   4.5075703e-05 3.8732949e-05]]]
1879
[[[5.3089960e-03

[[[6.4667134e-04 1.5140721e-06 2.5540590e-03 ... 2.4397512e-05
   3.9467301e-05 3.9374514e-05]]]
1378
[[[8.0263300e-04 2.1621124e-06 5.0678947e-03 ... 3.1566666e-05
   4.8938666e-05 4.6428544e-05]]]
1378
[[[2.1552618e-03 1.7093984e-06 1.5110371e-02 ... 2.7824019e-05
   4.4264711e-05 3.7880149e-05]]]
1879
[[[6.7533678e-03 1.6458081e-06 3.7418202e-02 ... 2.4782974e-05
   3.8829687e-05 3.2045082e-05]]]
2
-
Input sentence: after justin was given antiseizure medication
Decoded sentence: औरऔरके_END
[[[6.3888333e-04 1.4861835e-06 2.4685764e-03 ... 2.4063274e-05
   3.8897517e-05 3.8950227e-05]]]
1378
[[[7.336567e-04 2.184374e-06 4.514814e-03 ... 3.165542e-05 4.889799e-05
   4.682729e-05]]]
1378
[[[1.82871660e-03 1.72336115e-06 1.30157685e-02 ... 2.82093715e-05
   4.49260478e-05 3.85447529e-05]]]
1879
[[[5.4780245e-03 1.6128123e-06 3.2190979e-02 ... 2.5126628e-05
   3.9611441e-05 3.2572014e-05]]]
1879
[[[1.7503550e-02 1.6467704e-06 6.9531001e-02 ... 2.2271679e-05
   3.3702414e-05 2.7660843e-05]

[[[6.3938764e-04 1.5118810e-06 2.5294686e-03 ... 2.4354795e-05
   3.9390063e-05 3.9317947e-05]]]
1378
[[[7.8331912e-04 2.1688770e-06 4.9533108e-03 ... 3.1596792e-05
   4.8969625e-05 4.6538087e-05]]]
1378
[[[2.07291730e-03 1.71015245e-06 1.46761555e-02 ... 2.78804091e-05
   4.44118450e-05 3.80257443e-05]]]
1879
[[[6.4345440e-03 1.6372293e-06 3.6275581e-02 ... 2.4828185e-05
   3.8985676e-05 3.2185155e-05]]]
2
-
Input sentence: than when they arrived
Decoded sentence: औरऔरके_END
[[[7.8836735e-04 1.6824911e-06 3.4437478e-03 ... 2.6448448e-05
   4.2664811e-05 4.1787600e-05]]]
1378
[[[1.5043943e-03 2.0687285e-06 9.8441299e-03 ... 3.0690720e-05
   4.7805308e-05 4.4083845e-05]]]
1879
[[[5.1665031e-03 1.8522243e-06 3.0274203e-02 ... 2.6753820e-05
   4.1868097e-05 3.5970366e-05]]]
1879
[[[1.8920846e-02 1.7079180e-06 7.3411867e-02 ... 2.2315329e-05
   3.3670232e-05 2.8291257e-05]]]
2
-
Input sentence: nasa clean air study
Decoded sentence: औरकेके_END
[[[1.0130659e-02 9.7007169e-06 1.4432905e-02 .

...

[[[6.3570746e-04 1.4260269e-06 2.3601400e-03 ... 2.3342031e-05
   3.7679187e-05 3.8019094e-05]]]
1378
[[[6.3343818e-04 2.2196598e-06 3.6505186e-03 ... 3.1634539e-05
   4.8548896e-05 4.7455564e-05]]]
1378
[[[1.3136717e-03 1.7877002e-06 9.4848890e-03 ... 2.9057712e-05
   4.6299974e-05 4.0149582e-05]]]
1879
[[[3.5203190e-03 1.5565982e-06 2.3060629e-02 ... 2.5883663e-05
   4.1268318e-05 3.3635115e-05]]]
1879
[[[1.0624046e-02 1.6064993e-06 5.0320957e-02 ... 2.3523755e-05
   3.6262918e-05 2.9366074e-05]]]
2
-
Input sentence: and you have and entirely different story
Decoded sentence: औरऔरकेके_END
[[[6.3995825e-04 1.5010905e-06 2.5032328e-03 ... 2.4239358e-05
   3.9202474e-05 3.9183145e-05]]]
1378
[[[7.6258252e-04 2.1751109e-06 4.7631003e-03 ... 3.1642958e-05
   4.8990314e-05 4.6714529e-05]]]
1378
[[[1.9706597e-03 1.7150132e-06 1.3967622e-02 ... 2.8040566e-05
   4.4653927e-05 3.8273327e-05]]]
1879
[[[6.0293199e-03 1.6293020e-06 3.4538426e-02 ... 2.4986371e-05
   3.9289931e-05 3.2376160e-05]]]

[[[1.5775828e-03 1.7515518e-06 1.1300804e-02 ... 2.8633332e-05
   4.5601941e-05 3.9320850e-05]]]
1879
[[[4.5269090e-03 1.5919028e-06 2.7900560e-02 ... 2.5520849e-05
   4.0372706e-05 3.3115466e-05]]]
1879
[[[1.4228847e-02 1.6417902e-06 6.0952056e-02 ... 2.2878881e-05
   3.4878307e-05 2.8491266e-05]]]
2
-
Input sentence: more important than talking you will need to listen
Decoded sentence: औरऔरकेके_END
[[[1.2018757e-03 1.8523846e-06 5.4745912e-03 ... 2.8271947e-05
   4.5009860e-05 4.3563832e-05]]]
1378
[[[3.1408006e-03 2.0350340e-06 1.8916287e-02 ... 2.9129033e-05
   4.5177174e-05 4.1402935e-05]]]
1879
[[[1.2182864e-02 1.8526383e-06 5.5098724e-02 ... 2.4367167e-05
   3.7317674e-05 3.2252879e-05]]]
2
-
Input sentence: original foundation
Decoded sentence: औरके_END
[[[6.5464852e-04 1.5442610e-06 2.6386559e-03 ... 2.4758960e-05
   4.0082847e-05 3.9839095e-05]]]
1378
[[[8.7638682e-04 2.1442054e-06 5.6276745e-03 ... 3.1497118e-05
   4.8940074e-05 4.6125362e-05]]]
1378
[[[2.4973222e-03 1.70080

1378
[[[6.6674763e-04 2.2104614e-06 3.9937352e-03 ... 3.1681902e-05
   4.8809408e-05 4.7222933e-05]]]
1378
[[[1.5115036e-03 1.7557737e-06 1.0932603e-02 ... 2.8675398e-05
   4.5751196e-05 3.9401821e-05]]]
1879
[[[4.2675398e-03 1.5800658e-06 2.6786556e-02 ... 2.5533165e-05
   4.0562849e-05 3.3174987e-05]]]
1879
[[[1.3281106e-02 1.6338814e-06 5.8404960e-02 ... 2.2993097e-05
   3.5211342e-05 2.8694980e-05]]]
2
-
Input sentence: and we asked this woman
Decoded sentence: औरऔरकेके_END
[[[9.9571340e-04 1.7812762e-06 4.5085163e-03 ... 2.7563319e-05
   4.4165678e-05 4.2928143e-05]]]
1378
[[[2.3377347e-03 2.0415987e-06 1.4752520e-02 ... 2.9785764e-05
   4.6306828e-05 4.2532291e-05]]]
1879
[[[8.7432573e-03 1.8637166e-06 4.4248860e-02 ... 2.5343195e-05
   3.9183382e-05 3.3845157e-05]]]
2
-
Input sentence: for fun
Decoded sentence: औरके_END
[[[6.3227932e-04 1.4398422e-06 2.3744772e-03 ... 2.3495253e-05
   3.7948303e-05 3.8219827e-05]]]
1378
[[[6.4761640e-04 2.2164193e-06 3.8048790e-03 ... 3.1656116e

[[[3.1969205e-03 1.8211371e-06 2.0862145e-02 ... 2.7804641e-05
   4.3925877e-05 3.7545451e-05]]]
1879
[[[1.1147187e-02 1.7077659e-06 5.2662726e-02 ... 2.3791568e-05
   3.6668302e-05 3.0549912e-05]]]
2
-
Input sentence: other epics inspired by ramayana
Decoded sentence: औरकेके_END
[[[6.3470379e-04 1.4822022e-06 2.4556327e-03 ... 2.4008539e-05
   3.8818078e-05 3.8875165e-05]]]
1378
[[[7.2116888e-04 2.1891465e-06 4.4391789e-03 ... 3.1666306e-05
   4.8936545e-05 4.6905327e-05]]]
1378
[[[1.7768228e-03 1.7271313e-06 1.2714157e-02 ... 2.8275294e-05
   4.5067678e-05 3.8664406e-05]]]
1879
[[[5.2773762e-03 1.6095393e-06 3.1362757e-02 ... 2.5194366e-05
   3.9772160e-05 3.2678843e-05]]]
1879
[[[1.6814625e-02 1.6485103e-06 6.7842267e-02 ... 2.2389047e-05
   3.3944238e-05 2.7835864e-05]]]
2
-
Input sentence: and using this very simple machine
Decoded sentence: औरऔरकेके_END
[[[7.4280133e-03 2.9309283e-06 2.2983672e-02 ... 3.4175438e-05
   4.8907765e-05 4.7190606e-05]]]
2
-
Input sentence: read furthe

[[[4.0340926e-03 1.8342237e-06 2.5064548e-02 ... 2.7302307e-05
   4.2910320e-05 3.6825953e-05]]]
1879
[[[1.4467459e-02 1.7156691e-06 6.2237751e-02 ... 2.3111073e-05
   3.5252884e-05 2.9534067e-05]]]
2
-
Input sentence: instead artificial sweetners maybe used
Decoded sentence: औरकेके_END
[[[6.2911451e-04 1.4701282e-06 2.4199714e-03 ... 2.3848261e-05
   3.8539518e-05 3.8667742e-05]]]
1378
[[[6.8882992e-04 2.2033769e-06 4.2004366e-03 ... 3.1707135e-05
   4.8915110e-05 4.7116027e-05]]]
1378
[[[1.6303267e-03 1.7431303e-06 1.1765180e-02 ... 2.8510776e-05
   4.5452070e-05 3.9079903e-05]]]
1879
[[[4.7260053e-03 1.5956884e-06 2.8917514e-02 ... 2.5396823e-05
   4.0187751e-05 3.2967062e-05]]]
1879
[[[1.4890301e-02 1.6437046e-06 6.2840544e-02 ... 2.2725011e-05
   3.4613062e-05 2.8306651e-05]]]
2
-
Input sentence: was a sense of courage
Decoded sentence: औरऔरकेके_END
